In [ ]:
#wd<-"/home/jovyan/result/cellchat/time_domain_220325/celltype/binaried.permutate.Fisher_220423_cc/"
#wd<-"/home/jovyan/result/RCTD/fcreg1_220503/LR/220525/strategy1_220601/"
wd<-"/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/LR/220824.EM/"
if(!dir.exists(wd))
    dir.create(wd)
#wd<-"/home/jovyan/SCI_project/result/CellChat/neuron.astro_test/"
#dir.create(wd)
setwd(wd)
getwd()

In [ ]:
#library required packages
library(CellChat)
library(patchwork)
options(stringsAsFactors = FALSE)
library(Seurat)
#library(SPOTlight)
library(SingleCellExperiment)
#library(SpatialExperiment)
#library(scater)
#library(scran)
library(igraph)
library(stringr)
library(stringi)
library(reshape2)
library(RColorBrewer)

### get cellchat signaling gene list

In [ ]:
### extract cellchat LR interaction to our data
CellChatDB<-CellChatDB.mouse
interaction_input<-CellChatDB$interaction
head(interaction_input)

In [ ]:
cc_lr<-as.data.frame(str_split(interaction_input$interaction_name_2," - ",simplify = T))
cc_r<-as.data.frame(str_split(gsub("\\)","",gsub("\\(","",cc_lr$V2)),"\\+",simplify = T))
head(cc_lr)
head(cc_r)
colnames(cc_r)<-c("R1","R2")
cc_r$R1<-gsub("[[:space:]]","",cc_r$R1)
cc_r$R2<-gsub("[[:space:]]","",cc_r$R2,fixed = TRUE)
cc_lr<-cbind(cc_lr[,1],cc_r)
head(cc_lr)
cc_lr<-apply(cc_lr,2,function(x) gsub("[[:space:]]","",x))

In [ ]:
rownames(cc_lr)<-rownames(interaction_input)
colnames(cc_lr)<-c("ligand","receptor1","receptor2")
head(cc_lr)
tail(cc_lr)

In [45]:
write.csv(cc_lr,"cellchat.all.lr.interaction.df.csv")

In [ ]:
cc_lr<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/celltype/cellchat.all.lr.interaction.df.csv")
rownames(cc_lr)<-cc_lr[,1]
cc_lr<-cc_lr[,-1]
head(cc_lr)
dim(cc_lr)

In [ ]:
lr_genes<-unique(cc_lr[cc_lr!=""])
length(lr_genes)

### load spatial data

In [ ]:
ma<-readRDS("/home/jovyan/zxli_SCI/result/cellchat/WT.SCT.data.rds")
#ma<-readRDS("/home/jovyan/result/cellchat/time_domain_220325/celltype/binaried.permutate.Fisher_220509_cc/WT.SCT.scale.data.rds")
ma[1:5,1:5]

In [50]:
ma_bi<-ma

In [ ]:
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
head(meta)

In [26]:
test_me<-meta
test_me<-cbind(test_me,test2)

### load celltype dataframe

In [ ]:
#wt_df<-read.csv("/home/jovyan/result/RCTD/testC5/WT_all/fc1.fcreg1.conf5_220317/16celltypes.dec.meta.csv")
wt_df<-read.csv("/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/16celltypes.dec_conf.nor.meta.csv")
rownames(wt_df)<-wt_df[,1]
#colnames(wt_df)
celltypes<-colnames(wt_df)[c(47:ncol(wt_df))]
celltypes

weight_df<-wt_df[,celltypes]

#### remove Neuron and Oligodendrocyte noise through EM and get new wt_df -220601

In [17]:
test<-wt_df
test$Neuron_bi<-ifelse(test$Neuron>0.34405,1,0)
test$Oligo_bi<-ifelse(test$Oligodendrocyte>0.44393,1,0)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
ggplot(test,aes(x=x.ad,y=y.ad,color=as.factor(Neuron_bi)))+
    geom_point(size=1.2)+
    scale_color_manual(values = c("darkblue","yellow"))+
    theme(panel.background = element_blank(),
          axis.text = element_blank(),
          axis.title = element_blank(),
          axis.ticks = element_blank())
ggsave("Neuron.mean.binary.spatial.png",width = 20,height = 18,dpi = 300)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
ggplot(test,aes(x=x.ad,y=y.ad,color=as.factor(Oligo_bi)))+
    geom_point(size=1.2)+
    scale_color_manual(values = c("darkblue","yellow"))+
    theme(panel.background = element_blank(),
          axis.text = element_blank(),
          axis.title = element_blank(),
          axis.ticks = element_blank())
ggsave("Oligodendrocyte.mean.binary.spatial.png",width = 20,height = 18,dpi = 300)

In [ ]:
test$Neuron<-test$Neuron*test$Neuron_bi
test$Oligodendrocyte<-test$Oligodendrocyte*test$Oligo_bi
test<-test[,!colnames(test)%in%c("Neuron_bi","Oligo_bi")]
dim(test)
wt_df_ad<-test

In [ ]:
weight_df_ad<-wt_df_ad[,celltypes]
dim(weight_df_ad)

In [20]:
write.csv(weight_df_ad,"16celltypes.dec_conf.nor_NOrmMean.csv")

In [ ]:
weight_df_ad<-read.csv("../220530.EM/16celltypes.dec_conf.nor_NOrmMean.csv")
rownames(weight_df_ad)<-weight_df_ad[,1]
weight_df_ad<-weight_df_ad[,-1]
head(weight_df_ad)

In [ ]:
sub_lr_c<-intersect(lr_genes,rownames(ma))
length(sub_lr_c)

#### EM strategy -220530

In [28]:
#lr_bi_df<-data.frame(row.names = colnames(ma))
lr_bi_gm<-list()
for(i in 1:nrow(cc_lr)){
        lr<-as.character(cc_lr[i,][cc_lr[i,]!=""])
        if(all(lr %in% sub_lr_c)){
            temp<-ma[lr,]
            temp<-as.data.frame(apply(temp,2,function(x) exp(mean(log(x)))))
            colnames(temp)<-rownames(cc_lr)[i]
            s<-summary(temp[,1])
            em<-0.5*(s[[3]])+(s[[2]]+s[[5]])/4
            
            lr_bi_gm[[rownames(cc_lr)[i]]]<-em
            #binary
            #temp[,1]<-ifelse(temp[,1]>em,1,0)
            
            #lr_bi_df[,rownames(cc_lr)[i]]<-temp[,1]
            }
        }
                                      

In [24]:
saveRDS(lr_bi_gm,"SCT.data.LR.binary_GM.list.RDS")

In [59]:
saveRDS(lr_bi_df,"SCT.data.LR.binary.RDS")

In [ ]:
lr_bi_df<-readRDS("/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/LR/220530.EM/SCT.data.LR.binary.RDS")
dim(lr_bi_df)
head(lr_bi_df)

In [ ]:
### calculate lr average expression in each time -220825
me_exp<-readRDS("/home/jovyan/zxli_SCI/result/cellchat/celltype/2205013/spatial_visualization_220524/cellchat.lr.express.and.gm_express.meta.rds")

In [467]:
lr_time_mean<-data.frame(row.names = colnames(lr_bi_df))
for(i in unique(meta$time)){
    lr_time_mean[,i]<-0
    cells<-rownames(meta)[meta$time==i]
    for(j in rownames(lr_time_mean)){
        temp_m<-me_exp[cells,]
        s<-summary(temp_m[,j])
        em<-0.5*(s[[3]])+(s[[2]]+s[[5]])/4
        lr_time_mean[j,i]<-em
    }
}

In [526]:
lr_time_sum<-data.frame(row.names = colnames(lr_bi_df))
for(i in unique(meta$time)){
    lr_time_sum[,i]<-0
    cells<-rownames(meta)[meta$time==i]
    for(j in rownames(lr_time_sum)){
        temp_m<-me_exp[cells,]
        s<-sum(temp_m[,j])
        lr_time_sum[j,i]<-s
    }
}

In [528]:
write.csv(lr_time_sum,"lr.4times.sum.csv")

In [472]:
write.csv(lr_time_mean,"lr.4times.gm.csv")

In [ ]:
test_me<-meta
test_me<-cbind(test_me,lr_bi_df)
dim(test_me)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
ggplot(test_me,aes(x=x.ad,y=y.ad,color=as.factor(SPP1_CD44)))+
    geom_point(size=1.2)+
    #scale_color_manual(values = rev(c("orange","#911eb4","#e6beff","blue","grey")))+
    theme(panel.background = element_blank(),
          axis.text = element_blank(),
          axis.title = element_blank(),
          axis.ticks = element_blank())+
    scale_color_manual(values = c("#440154FF","#FDE725FF"))
ggsave("SPP1_CD44.binary.spatial.png",width = 20,height = 18,dpi = 300)

In [11]:
me_exp<-readRDS("/home/jovyan/zxli_SCI/result/cellchat/celltype/2205013/spatial_visualization_220524/cellchat.lr.express.and.gm_express.meta.rds")

### firstly calculate the lr gm across 16 samples -220825

In [ ]:
#subset represent samples
meta_sub<-subset(meta,sample %in% c('WT_sham_H_R2_1mm_2','WT_sham_H_R2_4','WT_sham_T_210323_4','WT_sham_T_R2_1mm_4',
                                                  'WT_3h_H_R2_1mm_2','WT_3h_H_R2_2','WT_3h_T_R2_1','WT_3h_T_210330_1mm_3',
                                                  'WT_24h_H_R1_1mm_3','WT_24h_H_R1_4','WT_24h_T_201231_3','WT_24h_T_R1_1mm_3',
                                                  'WT_72h_H_R1_1mm_2','WT_72h_H_210323_1','WT_72h_T_R2_4','WT_72h_T_R1_1mm_3'))
dim(meta_sub)
cells<-rownames(meta_sub)


In [ ]:
ma_sub<-ma[sub_lr_c,cells]
dim(ma_sub)

In [5]:
lr_bi_df<-readRDS("../220530.EM/SCT.data.LR.binary.RDS")

### test celltype pair with lr each time point

1. firstly check the lr binary spatial distribution 

In [ ]:
test_me<-cbind(meta_sub,temp_lr_bi_df)
dim(test_me)

In [ ]:
options(repr.plot.width=20,repr.plot.height=18)
ggplot(test_me,aes(x=x.ad,y=y.ad,color=as.factor(PSAP_GPR37L1)))+
    geom_point(size=1.2)+
    #scale_color_manual(values = rev(c("orange","#911eb4","#e6beff","blue","grey")))+
    theme(panel.background = element_blank(),
          axis.text = element_blank(),
          axis.title = element_blank(),
          axis.ticks = element_blank())+
    scale_color_manual(values = c("#440154FF","#FDE725FF"))
ggsave("PSAP_GPR37L1.binary.spatial.png",width = 20,height =18,dpi = 300 )

2. fisher exact test on lr and cp

In [6]:
num_talk<-apply(lr_bi_df,2,sum)

In [7]:
# remove num_talk <20                           
lr_bi_df_sub<-lr_bi_df[,num_talk>20]

In [20]:
cal_jc<-function(weight_df){
    df<-data.frame(row.names = colnames(weight_df))
    for(i in colnames(weight_df)){
        df[,i]<-0
        for(j in colnames(weight_df)){
            jc<-sum(weight_df[,i]>0 & weight_df[,j]>0)/sum(weight_df[,i]>0 | weight_df[,j]>0)
            df[j,i]<-jc
        }
    }
    return(df)
}

In [ ]:
for(s in unique(meta_sub$time)){
#for(s in c('WT_sham_H_R2_1mm_3','WT_sham_H_R2_1mm_2','WT_sham_H_R2_1mm_1','WT_sham_H_R2_1mm_4')){
   ### construct expression matrix for each section
    #s<-"WT_sham_H_R2_1mm_2"
    cell<-meta_sub[meta_sub$time==s,"barcode"]
    temp_lr_bi_df<-lr_bi_df_sub[cell,]                           
    #cell-pair matrix
    temp_wt<-weight_df_ad[cell,]#weight_df[cell,]
    temp_wt<-temp_wt[,colSums(temp_wt)>1] #remove un-existed cell types
    wt_bi_df<-data.frame(row.names = rownames(temp_wt))
    cor_df<-cor(temp_wt,method = "spearman")
    #jc_df<-cal_jc(temp_wt)
    # construct cell type pairs
    ij <- combn(colnames(temp_wt), 2,simplify = T)
    length(ij)
    for(i in 1:ncol(ij)){
        pair<-ij[,i]
        cn<-paste(pair[1],pair[2],sep = "_")
        if(cor_df[pair[1],pair[2]]<(-0.4) & s %in% c("WT_sham","WT_3h")){
            wt_bi_df[,cn]<-0
            }
        #if(jc_df[pair[1],pair[2]]<0.06){
        #    wt_bi_df[,cn]<-0
        #    }
        else{
            inter_spots1<-rownames(temp_wt[temp_wt[,pair[1]]>0 & temp_wt[,pair[2]]>0,])
            wt_bi_df[,cn]<-0
            if(length(inter_spots1)>10)
                wt_bi_df[inter_spots1,cn]<-1  
            }
              
        }
    head(wt_bi_df)
    f2<-"cell-pair.colocalize.binary_result2/"
    if(!dir.exists(f2))
        dir.create(f2)
    write.csv(wt_bi_df,paste0(f2,s,".binaried.celltype_pair.csv"))
    cp_bi_df_sub<-wt_bi_df[,colSums(wt_bi_df)!=0]
    
    ###Fisher's exact test
    #calculate LR's association with cellpair and get the LR-cp double sig spot number
    lr_cp_pval_df<-data.frame(row.names = colnames(temp_lr_bi_df))
    lr_cp_spot_n<-data.frame(row.names = colnames(temp_lr_bi_df))
    lr_cp_jc_df<-data.frame(row.names = colnames(temp_lr_bi_df))
    #lr_cp_pcc_df<-data.frame(row.names = colnames(temp_lr_bi_df))                          
    for(i in colnames(cp_bi_df_sub)){
        lr_cp_pval_df[,i]<-NA
        lr_cp_spot_n[,i]<-NA
        for(j in colnames(temp_lr_bi_df)){
            #if(all(table(temp_temp_lr_bi_df[,j],cp_bi_df_sub[,i])==2)){
            pp1<-temp_lr_bi_df[,j]
            lr_ind<-rownames(temp_lr_bi_df)[pp1==1]
            pp2<-cp_bi_df_sub[,i]
            cp_ind<-rownames(cp_bi_df_sub)[pp2==1]
            # pcc<-cor(pp1,pp2)
            pp1<-factor(pp1,levels=c(0,1))
            pp2<-factor(pp2,levels=c(0,1))
            f_df<-table(pp1,pp2)
            p<-fisher.test(f_df,alternative = "greater")$p.val
            lr_cp_pval_df[j,i]<-p
            
            n<-f_df[2,2]
            #nn<-f_df[1,1]
            lr_cp_spot_n[j,i]<-n
            
            #JC<-length(intersect(cp_ind,lr_ind))/length(unique(c(cp_ind,lr_ind)))
            #lr_cp_jc_df[j,i]<-JC
           # lr_cp_pcc_df[j,i]<-pcc
            #f(n<nn)
            #   lr_cp_pval_df[j,i]<-1
            #}
            
        }
    }
    
    lr_cp_n_long<-lr_cp_spot_n
    lr_cp_n_long$lr<-rownames(lr_cp_n_long)
    lr_cp_n_long<-melt(lr_cp_n_long)
    colnames(lr_cp_n_long)<-c("lr","cp","n_spot")
    lr_cp_n_long$key<-paste0(lr_cp_n_long$lr,"-",lr_cp_n_long$cp)
    head(lr_cp_n_long)
    ## add the most possible cp of each LR and its pval
    #r_cp_pval_df$top1.cp<-colnames(lr_cp_pval_df)[apply(lr_cp_pval_df,1,which.min)]
    #olnames(lr_cp_pval_df)
    #r_cp_pval_df$top1.cp.pval<-apply(lr_cp_pval_df[,1:(ncol(lr_cp_pval_df)-1)],1,min)
    ##ead(lr_cp_pval_df)
    f3<-"LR.cp.Fisher.pval_result2/"
    if(!dir.exists(f3))
        dir.create(f3)
    f3a<-"LR.cp.Fisher.pval_result2/pval.df/"
    if(!dir.exists(f3a))
        dir.create(f3a)                       
    write.csv(lr_cp_pval_df,paste0(f3a,s,".cellpair.LR.fishers.pval.df"))
    
    lr_cp_long_df<-lr_cp_pval_df#[,-which(colnames(lr_cp_pval_df)%in%c("top1.cp","top1.cp.pval"))]
    lr_cp_long_df$lr<-rownames(lr_cp_long_df)
    lr_cp_long_df<-melt(lr_cp_long_df)
    lr_cp_long_df$key<-paste0(lr_cp_long_df$lr,"-",lr_cp_long_df$variable)
    colnames(lr_cp_long_df)<-c("lr","cp","pval","key")
    head(lr_cp_long_df)
    
    #jc_long_df<-lr_cp_jc_df                           
    #jc_long_df$lr<-rownames(lr_cp_jc_df)
    #jc_long_df<-melt(jc_long_df)
    #colnames(jc_long_df)<-c("lr",'cp','JC')
    #jc_long_df$key<-paste0(jc_long_df$lr,"-",jc_long_df$cp)
    #head(jc_long_df)
    
   #pcc_long_df<-lr_cp_pcc_df                  --\++     
   #pcc_long_df$lr<-rownames(pcc_long_df)
   #pcc_long_df<-melt(pcc_long_df)
   #colnames(pcc_long_df)<-c("lr","cp","pcc")
   #pcc_long_df$key<-paste0(jc_long_df$lr,"-",jc_long_df$variable)
   #head(jc_long_df)                           
    
    lr_cp_long<-merge(lr_cp_long_df,lr_cp_n_long[,c(3,4)],by = "key")
    #lr_cp_long<-merge(lr_cp_long,jc_long_df[,c(3,4)],by = "key")
    #lr_cp_long<-merge(lr_cp_long,jc_long_df[,c(3,4)],by = "key")                           
    head(lr_cp_long)
    dim(lr_cp_long)
    
    lr_cp_long_sub<-subset(lr_cp_long,n_spot>10&pval<0.01)
    #calculate cp's lr average expression
    lr_cp_long_sub$EM<-0
    lr_cp_long_sub$cp_lr_mean<-0
    for(i in unique(lr_cp_long_sub$cp)){
        cp_spots<-rownames(cp_bi_df_sub)[cp_bi_df_sub[,i]==1]
        cp_lr<-unique(lr_cp_long_sub[lr_cp_long_sub$cp==i,"lr"])
        for(j in cp_lr){
            cp_lr_MExp<-mean(me_exp[cp_spots,j])
            lr_cp_long_sub[lr_cp_long_sub$cp==i & lr_cp_long_sub$lr==j,"EM"]<-lr_bi_gm[[j]]
            lr_cp_long_sub[lr_cp_long_sub$cp==i & lr_cp_long_sub$lr==j,"cp_lr_mean"]<-cp_lr_MExp
        }
    }
    f3b<-"LR.cp.Fisher.pval_result2/pval.long.df/"
    if(!dir.exists(f3b))
        dir.create(f3b) 
    write.csv(lr_cp_long,paste0(f3b,s,".cellpair.LR.pval.spots.long.all.csv"))
    write.csv(lr_cp_long_sub,paste0(f3b,s,".cellpair.LR.pval.spots.long.csv"))            
}                   

#### just calculate sample1, and directly get the result of each timepoint

In [ ]:
f3b<-"LR.cp.Fisher.pval_result2/pval.long.df/"
f<-dir(f3b)
f<-f[c(2,4,6,8)]
f

In [ ]:
temp1<-read.csv(paste0(f3b,f[4]))
temp1<-temp1[,-1]
temp1<-subset(temp1,pval<1e-3&n_spot>10)
dim(temp1)
head(temp1)

In [ ]:
# load sham data
temp1<-read.csv(paste0(f3b,f[4]))
temp1<-temp1[,-1]
temp1<-subset(temp1,pval<=1e-3&n_spot>10)
#colnames(temp1)<-c("key","lr","cp","pval","n_spot","EM")
temp1$autocrine<-c(str_split(temp1$lr,"_",simplify = T)[,1]==str_split(temp1$lr,"_",simplify = T)[,2])
head(temp1)
temp2<-read.csv(paste0(f3b,f[2]))
temp2<-temp2[,-1]
temp2<-subset(temp2,pval<=1e-3&n_spot>10)
#colnames(temp2)<-c("key","lr","cp","pval","n_spot","JC")
temp2$autocrine<-c(str_split(temp2$lr,"_",simplify = T)[,1]==str_split(temp2$lr,"_",simplify = T)[,2])
head(temp2)
temp3<-read.csv(paste0(f3b,f[1]))
temp3<-temp3[,-1]
temp3<-subset(temp3,pval<=1e-3&n_spot>10)
#colnames(temp3)<-c("key","lr","cp","pval","n_spot","JC")
temp3$autocrine<-c(str_split(temp3$lr,"_",simplify = T)[,1]==str_split(temp3$lr,"_",simplify = T)[,2])
head(temp3)
temp4<-read.csv(paste0(f3b,f[3]))
temp4<-temp4[,-1]
temp4<-subset(temp4,pval<=1e-3&n_spot>10)
#colnames(temp4)<-c("key","lr","cp","pval","n_spot","JC")
temp4$autocrine<-c(str_split(temp4$lr,"_",simplify = T)[,1]==str_split(temp4$lr,"_",simplify = T)[,2])
head(temp4)

In [6]:
sf3<-"4time.spmc-04.pval1e-4_spot10.lr.cp_sc2/"
    if(!dir.exists(sf3))
        dir.create(sf3)

In [ ]:
# add crosstalk shared with single cell result at corresponding timepoint
sc_df<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/Astr_state_220623/uninj.fc05.pc05.min10.interaction.net.csv")
sc_df$source[sc_df$source %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$target[sc_df$target %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
head(sc_df)
sc_int<-intersect(unique(temp1$key),unique(c(sc_df$key1,sc_df$key2)))
sc_int                   
temp1$sc_shared<-"N"
temp1$sc_shared<-ifelse(temp1$key %in% sc_int,"Y","N")
head(temp1)
table(temp1$sc_shared)

write.csv(temp1,paste0(sf3,"sham.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))     

In [ ]:
# add crosstalk shared with single cell result at corresponding timepoint
#sc_df<-read.csv("/home/jovyan/result/cellchat/time_domain_220325/10X_sc/allcelltype_result/uninj_s3.df.net.csv")
#sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
#sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
#head(sc_df)
#sc_int<-intersect(unique(temp2$key),unique(c(sc_df$key1,sc_df$key2)))
#sc_int                   
temp2$sc_shared<-"N"
#temp2$sc_shared<-ifelse(temp2$key %in% sc_int,"Y","N")
head(temp2)
table(temp2$sc_shared)

write.csv(temp2,paste0(sf3,"3h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))     

In [ ]:
# add crosstalk shared with single cell result at corresponding timepoint
sc_df<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/Astr_state_220623/d1.fc05.pc05.min10.interaction.net.csv")
sc_df$source[sc_df$source %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$target[sc_df$target %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
head(sc_df)
sc_int<-intersect(unique(temp3$key),unique(c(sc_df$key1,sc_df$key2)))
sc_int                   
temp3$sc_shared<-"N"
temp3$sc_shared<-ifelse(temp3$key %in% sc_int,"Y","N")
head(temp3)
table(temp3$sc_shared)

write.csv(temp3,paste0(sf3,"24h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))     

In [ ]:
sc_df<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/Astr_state_220623/d1.fc05.pc05.min10.interaction.net.csv")
sc_df$source[sc_df$source %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$target[sc_df$target %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
head(sc_df)

In [ ]:
# add crosstalk shared with single cell result at corresponding timepoint
sc_df<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/Astr_state_220623/d3.fc05.pc05.min10.interaction.net.csv")
sc_df$source[sc_df$source %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$target[sc_df$target %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
head(sc_df)
sc_int<-intersect(unique(temp4$key),unique(c(sc_df$key1,sc_df$key2)))
sc_int                   
temp4$sc_shared<-"N"
temp4$sc_shared<-ifelse(temp4$key %in% sc_int,"Y","N")
head(temp4)
table(temp4$sc_shared)

write.csv(temp4,paste0(sf3,"72h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))     

In [ ]:
sc_df<-read.csv("/home/jovyan/zxli_SCI/result/cellchat/Astr_state_220623/d3.fc05.pc05.min10.interaction.net.csv")
sc_df$source[sc_df$source %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$target[sc_df$target %in% c("WM.Gfap","GM.Gfap")]<-"Astrocyte.Gfap"
sc_df$key1<-paste0(sc_df$interaction_name,"-",sc_df$source,"_",sc_df$target)
sc_df$key2<-paste0(sc_df$interaction_name,"-",sc_df$target,"_",sc_df$source)
head(sc_df)
sc_int<-intersect(unique(temp4$key),unique(c(sc_df$key1,sc_df$key2)))
sc_int 

### 4 time merged

In [ ]:
temp1<-read.csv(paste0(sf3,"sham.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))
temp1<-temp1[,-1]
head(temp1)

In [ ]:
temp2<-read.csv(paste0(sf3,"3h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))
temp2<-temp2[,-1]
head(temp2)

In [ ]:
temp3<-read.csv(paste0(sf3,"24h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))
temp3<-temp3[,-1]
head(temp3)

In [ ]:
temp4<-read.csv(paste0(sf3,"72h.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))
temp4<-temp4[,-1]
head(temp4)

In [11]:
temp1$time<-"sham"
temp2$time<-"3h"
temp3$time<-"24h"
temp4$time<-"72h"

In [ ]:
temp<-Reduce(rbind,list(temp1,temp2,temp3,temp4))
dim(temp)

In [23]:
### centralize each lr
temp_scale<-temp
temp_scale$cp_lr_mean_scaled<-0

for(i in unique(temp$lr)){
    lr_temp<- temp_scale[temp_scale$lr==i,]
    lr_min<-min(lr_temp$cp_lr_mean)
    lr_max<-max(lr_temp$cp_lr_mean)
    lr_em<-unique(temp_scale[temp_scale$lr==i,"EM"])
    temp_scale[temp_scale$lr==i,"cp_lr_mean_scaled"]<-(temp_scale[temp_scale$lr==i,"cp_lr_mean"]-lr_em)/(lr_max-lr_em)
}

In [24]:
temp<-temp_scale

In [217]:
write.csv(temp,paste0(sf3,"4times.merged.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))

In [ ]:
temp<-read.csv(paste0(sf3,"4times.merged.spmc-04_2.pval1e-3_spot10.lr.cp.long.df_sc.csv"))
rownames(temp)<-temp[,1]
temp<-temp[,-1]
head(temp)

### Visualize

In [38]:
sf2<-"geompoint.cp_spmc-04_2.all_top3.lr.png2/"
if(!dir.exists(sf2))
    dir.create(sf2)

In [6]:
#remove Dendritic and Astro.Svep1
temp<-temp[!grepl("Astro.Svep1",temp$cp),]

In [ ]:
### adjust 220725
unique(temp$cp)
temp$cp<-factor(temp$cp,levels = c("Astrocyte.Slc7a10_Neuron","Astrocyte.Gfap_Neuron","Neuron_Vascular",
                                  "Homeostatic.Microglia_Neuron","Neuron_R.Microglia","Neuron_OPC","Neuron_Oligodendrocyte",
                                   "Astrocyte.Gfap_Oligodendrocyte","Oligodendrocyte_Vascular",'Oligodendrocyte_Ependymal',"Fibroblast_Oligodendrocyte",
                                   "Homeostatic.Microglia_Oligodendrocyte","Oligodendrocyte_R.Microglia","Oligodendrocyte_Myeloid",
                                   "Monocyte_Oligodendrocyte",'Neutrophil_Oligodendrocyte',"Oligodendrocyte_OPC","Oligodendrocyte_Macrophage","Astrocyte.Gfap_Vascular",
                                   "Astrocyte.Slc7a10_Vascular","Astrocyte.Gfap_Fibroblast","Astrocyte.Gfap_Homeostatic.Microglia",
                                   "Astrocyte.Gfap_R.Microglia","Astrocyte.Gfap_Myeloid","Astrocyte.Slc7a10_OPC","Astrocyte.Gfap_OPC",
                                   "Astrocyte.Gfap_Neutrophil","Astrocyte.Gfap_Monocyte","Astrocyte.Gfap_Ependymal","Astrocyte.Gfap_Macrophage","Fibroblast_R.Microglia",
                                   "Vascular_Macrophage","Fibroblast_Macrophage","OPC_Macrophage","Fibroblast_OPC",'Fibroblast_Monocyte','Vascular_OPC','OPC_R.Microglia'
                                  )
               )

In [8]:
temp$time<-factor(temp$time,levels = c("sham","3h","24h","72h"))
temp<-arrange(temp,time)
temp$group<-paste0(temp$time,"-",temp$cp)
temp$group<-factor(temp$group,levels = unique(temp$group))

In [ ]:
top5<-temp %>% filter(autocrine==FALSE)%>%
    group_by(group) %>% top_n(2,wt = -pval) %>%  top_n(2,wt=n_spot) %>% top_n(2,wt=EM) 
dim(top5)
unique(top5$lr)

In [ ]:
top5<-temp[temp$lr%in%unique(top5$lr),]
#top5$lr<-factor(top5$lr,levels = unique(top5$lr))
head(top5)

In [541]:
### directly look at the spatial pattern
fd<-"cp.top2.lr.spatial/"
if(!dir.exists(fd))
    dir.create(fd)
for(i in lr_order){
    options(repr.plot.width=20,repr.plot.height=18)
    ggplot(me_exp,aes_string(x='x.ad',y='y.ad',color=i))+
        geom_point(size=1.2)+
        #scale_color_manual(values = rev(c("orange","#911eb4","#e6beff","blue","grey")))+
        theme(panel.background = element_blank(),
              axis.text = element_blank(),
              axis.title = element_blank(),
              axis.ticks = element_blank())+
        scale_color_gradientn(colours =c("#DCDCDC","#A52A2A"))
        #scale_color_manual(values = c("#d6d4e0","#c83349"))
        #scale_color_manual(values = c('#f2e394','#d5f4e6','#80ced6','#eea29a','#f9d5e5'#'#5b9aa0'#
         #                             ,'#618685'#,
                                      
         #                             ,'#d6cbd3','#a2836e'
         #                             ))
    ggsave(paste0(fd,i,".SCT.data.GM.spatial.png"),width = 20,height = 18,dpi = 400)
}

In [11]:
### manually check throught time trend
lr_order<-c('EFNB3_EPHB1','PSAP_GPR37','SEMA4D_PLXNB3','SEMA4D_PLXNB1','PTN_PTPRZ1','PTN_SDC3',
            'FGF1_FGFR1','FGF1_FGFR3','FGF1_FGFR2','FGF9_FGFR3','GAS6_TYRO3','NFASC_CNTN1_CNTNAP1','PENK_OPRL1',#decrease
            'NCAM1_L1CAM','NRXN1_NLGN3','NRXN3_NLGN1','NRXN3_NLGN2',
            'NRXN3_NLGN3','NRXN2_NLGN3','FGF9_FGFR1', #switch
            'PSAP_GPR37L1','CXCL12_ACKR3','ADCYAP1_ADCYAP1R1','IL1B_IL1R2',
            'GRN_SORT1','CSF1_CSF1R','PDGFA_PDGFRA','C3_C3AR1',
            'SEMA6D_PLXNA1_TREM2','BDNF_NTRK2','SEMA6D_PLXNA1',
            'COL1A1_SDC4','COL6A2_ITGAV_ITGB8',
            'COL1A2_CD44','COL1A1_CD44','COL1A2_SDC4','COL6A2_SDC4','COL4A1_SDC4',
            'COL9A3_CD44','COL6A3_CD44','COL6A3_SDC1','COL6A3_SDC4','COL4A2_SDC4',
            'LAMB2_SV2C','LAMB2_CD44',
            'FN1_SDC1','FN1_CD44','FN1_ITGAV_ITGB1','FN1_ITGA5_ITGB1',
            'ANGPTL4_CDH5','ANGPTL4_SDC4',
            'SPP1_CD44','SPP1_ITGA5_ITGB1','SPP1_ITGAV_ITGB5',
            'THBS1_CD36','THBS1_SDC4','THBS1_CD47','THBS1_SDC1','THBS2_SDC4',
            'TNC_SDC4','TNC_SDC1',
            'TNR_SDC1','TNR_SDC4' #increase
            )

In [12]:
top5$lr<-factor(top5$lr,levels = lr_order)

In [15]:
options(repr.plot.width=25,repr.plot.height=13)
p<-ggplot(top5,aes(x=cp,y=lr))+
    geom_point(aes(size=-log10(pval),color=cp_lr_mean_scaled,shape=sc_shared
                  ))+
    scale_color_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 7,name = "Spectral")))(20)#c("#8227d8","#f7c5c4","#e85957"
                                    #  ,"#e00512")
                         )+ 
    theme_bw()+#ggtitle(label = "            sham                                   3h                                   24h                                             72h")+
    theme(axis.text.x = element_text(angle = 60,size = 12,vjust = 1,hjust = 1),
          axis.text.y.left = element_text(angle = 30,size = 12),axis.title = element_blank(),
         strip.text=element_text(color="white")          
         )+
    facet_grid(~time)
    

In [ ]:
g <- ggplot_gtable(ggplot_build(p))
stripr <- which(grepl('strip-t', g$layout$name))
fills <- c("#374E55FF","#DF8F44FF","#00A1D5FF","#B24745FF")
k <- 1
for (i in stripr) {
  j <- which(grepl('rect', g$grobs[[i]]$grobs[[1]]$childrenOrder))
  g$grobs[[i]]$grobs[[1]]$children[[j]]$gp$fill <- fills[k]
  k <- k+1
}
grid::grid.draw(g)

In [ ]:
pdf(paste0(sf2,"alltime.pval1e-3.n10.cp.top2pval.nspot_220825_sc.pdf"),width = 25,height = 13)
grid::grid.draw(g)
#ggsave(paste0(sf2,"alltime.pval005.n10.jc005.cp.top2pval.nspot_220725.png"),width = 25,height = 13,dpi = 300)
dev.off()

In [ ]:
#### just show switch LR -221011
lr<-c('NCAM1_L1CAM','NRXN1_NLGN3','NRXN3_NLGN1','NRXN3_NLGN2','NRXN3_NLGN3','NRXN2_NLGN3','FGF9_FGFR1')
#cp<-c("Astrocyte.Slc7a10_Neuron","Astrocyte.Gfap_Neuron","Neuron_Vascular","Homeostatic.Microglia_Neuron","Neuron_R.Microglia","Neuron_OPC","Neuron_Oligodendrocyte")
top5_sub<-top5[ top5$lr %in% lr,]
dim(top5_sub)

In [36]:
options(repr.plot.width=7,repr.plot.height=4)
p<-ggplot(top5_sub,aes(x=cp,y=lr))+
    geom_point(aes(size=-log10(pval),color=cp_lr_mean_scaled#,shape=sc_shared
                  ))+
    scale_color_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 7,name = "Spectral")))(20)#c("#8227d8","#f7c5c4","#e85957"
                                    #  ,"#e00512")
                         )+ 
    theme_bw()+#ggtitle(label = "            sham                                   3h                                   24h                                             72h")+
    theme(axis.text.x = element_text(angle = 60,size = 12,vjust = 1,hjust = 1),
          axis.text.y.left = element_text(angle = 30,size = 12),axis.title = element_blank(),
         strip.text=element_text(color="white")          
         )+
    facet_grid(~time)
    

In [ ]:
g <- ggplot_gtable(ggplot_build(p))
stripr <- which(grepl('strip-t', g$layout$name))
fills <- c("#374E55FF","#DF8F44FF","#00A1D5FF","#B24745FF")
k <- 1
for (i in stripr) {
  j <- which(grepl('rect', g$grobs[[i]]$grobs[[1]]$childrenOrder))
  g$grobs[[i]]$grobs[[1]]$children[[j]]$gp$fill <- fills[k]
  k <- k+1
}
grid::grid.draw(g)

In [ ]:
pdf(paste0(sf2,"alltime.pval1e-3.n10.cp.top2pval.nspot_221011_switchLR.pdf"),width = 7,height = 4)
grid::grid.draw(g)
#ggsave(paste0(sf2,"alltime.pval005.n10.jc005.cp.top2pval.nspot_220725.png"),width = 25,height = 13,dpi = 300)
dev.off()

### astrocyte display -> each cell type 220519

In [42]:
#sf<-"Astr.Gfap_top5lr.png/"
sf<-"eachct_top3lr.png/"
if(!dir.exists(sf))
    dir.create(sf)

In [ ]:
temp$c1<-str_split(temp$cp,"_",simplify = T)[,1]
temp$c2<-str_split(temp$cp,"_",simplify = T)[,2]
head(temp)

In [ ]:
ct<-unique(c(temp$c1,temp$c2))
ct

In [ ]:
#### manual adjust the figure scale
i='Astrocyte.Gfap'
test<-temp
    test<-test[grepl(i,test$cp),]
    dim(test)
    #test$c1<-str_split(test$cp,"_",simplify = T)[,1]
    #test$c2<-str_split(test$cp,"_",simplify = T)[,2]
    test$target<-ifelse(test$c1==i,test$c2,test$c1)
    test$group<-paste0(test$time,"_",test$target)
    test$group<-factor(test$group,levels = unique(test$group))
    head(test)
    levels(test$group)
    #write.csv(test,paste0(sf,i,".lr.pval001.jc01.spot10.csv"))
    top5<-test %>% filter(autocrine==FALSE)%>% 
        group_by(group) %>% top_n(5,wt = -pval) %>% top_n(5,wt = JC) #%>% top_n(5,wt = n_spot)
    dim(top5)
    unique(top5$lr)
    ym<-length(unique(top5$lr))+0.5
    ym
    x1<-length(grep("sham_",unique(top5$group)))+0.5
    x1
    x2<-length(grep("3h_",unique(top5$group)))+x1
    x2
    x3<-length(grep("24h_",unique(top5$group)))+x2
    x3
    x4<-length(grep("72h_",unique(top5$group)))+x3
    x4
    top5<-arrange(top5,group)
    top5$lr<-factor(top5$lr,levels = unique(top5$lr))
    


In [ ]:
unique(top5$lr)
head(test)
top5<-test[test$lr %in% as.character(unique(top5$lr)),]
top5$lr<-factor(top5$lr,levels = unique(top5$lr))

In [ ]:
test<-data.frame(row.names = unique(top5$lr))
for(k in unique(top5$group)){
    test[,k]<-0
    for(j in rownames(test)){
        if(nrow(top5[top5$lr==j&top5$group==k,])>0)
            test[j,k]<-1
    }
    
    
}
h<-hclust(dist(x=test,method = "binary"))
length(h$order)
lr_order<-rownames(test)[h$order]
head(lr_order)

In [ ]:
test<-test[lr_order,]
test_sham<-test[,grep("sham",colnames(test))]
test_3h<-test[,grep("3h",colnames(test))]
test_24h<-test[,grep("24h",colnames(test))]
test_72h<-test[,grep("72h",colnames(test))]
h1<-hclust(dist(t(test_sham),method = "binary"))
colnames(test_sham)[h1$order]
h2<-hclust(dist(t(test_3h),method = "binary"))
colnames(test_3h)[h2$order]
h3<-hclust(dist(t(test_24h),method = "binary"))
colnames(test_24h)[h3$order]
h4<-hclust(dist(t(test_72h),method = "binary"))
colnames(test_72h)[h4$order]

In [100]:
top5$lr<-factor(top5$lr,levels = rev(lr_order))
top5$group<-factor(top5$group,levels = c(rev(colnames(test_sham)[h1$order]),
                                         rev(colnames(test_3h)[h2$order]),
                                         rev(colnames(test_24h)[h3$order]),
                                         rev(colnames(test_72h)[h4$order])))

In [ ]:
wd<-8
ht<-13
options(repr.plot.width=wd,repr.plot.height=ht)
    ggplot(top5,aes(x=group,y=lr))+
        geom_point(aes(size=JC,color=-log10(pval),shape=sc_shared
                      ))+
        scale_color_gradientn(colours = c("#8227d8","#f7c5c4","#e85957"
                                          ,"#e00512"))+ 
        theme_bw()+#ggtitle(label = "            sham                3h             24h               72h")+
        theme(axis.text.x = element_text(angle = 60,size = 12,vjust = 1,hjust = 1),
              axis.text.y.left = element_text(angle = 30,size = 12),axis.title = element_blank())+
        annotate("rect",xmin=.5,xmax = x1,ymin = 0,ymax = ym,alpha=.1,color="#374E55FF",fill="#374E55FF")+
        annotate("rect",xmin=x1,xmax = x2,ymin = 0,ymax = ym,alpha=.1,color="#DF8F44FF",fill="#DF8F44FF")+
        annotate("rect",xmin=x2,xmax = x3,ymin = 0,ymax = ym,alpha=.1,color="#00A1D5FF",fill="#00A1D5FF")+
        annotate("rect",xmin=x3,xmax = x4,ymin = 0,ymax = ym,alpha=.1,color="#B24745FF",fill="#B24745FF")#+
    
    
    #+
        #facet_wrap(~time,nrow = 1,ncol = 4)
    
   ggsave(paste0(sf,i,".alltime.pval005.n15.jc005.cp.top5pval.jc.lr_sc.220607.png"),width =wd,height = ht,dpi = 400)

### network

In [3]:
sf<-"igraph.cellinteraction.number2/"
if(!dir.exists(sf))
    dir.create(sf)

In [4]:
geometric.mean<-function(x) {x=exp(mean(log(x)))
                            return(x)
                            }

In [ ]:
temp1<-temp[temp$time=="sham",]
head(temp1)
temp2<-temp[temp$time=="3h",]
head(temp2)
temp3<-temp[temp$time=="24h",]
head(temp3)
temp4<-temp[temp$time=="72h",]
head(temp4)

In [ ]:
sn1<-as.data.frame(table(temp1$cp))
sn1$source<-str_split(sn1$Var1,"_",simplify = T)[,1]
sn1$target<-str_split(sn1$Var1,"_",simplify = T)[,2]
rownames(sn1)<-sn1[,1]
sn1<-sn1[,-1]
colnames(sn1)<-c("value","from","to")
sn1<-sn1[,c(2,3,1)]
sn1$pval<- -log10(aggregate(temp1$pval,list(temp1$cp),FUN=geometric.mean)[,2])
sn1$n_spot<- round(aggregate(temp1$n_spot,list(temp1$cp),FUN=mean)[,2],2)
sn1$exp<-round(aggregate(temp1$cp_lr_mean_scaled,list(temp1$cp),FUN=mean)[,2],2)
head(sn1)

In [310]:
write.csv(sn1,paste0(sf,"sham.spmc-04.pval1e-3_spot10.igraph.df.csv"))

In [ ]:
sn2<-as.data.frame(table(temp2$cp))
sn2$source<-str_split(sn2$Var1,"_",simplify = T)[,1]
sn2$target<-str_split(sn2$Var1,"_",simplify = T)[,2]
rownames(sn2)<-sn2[,1]
sn2<-sn2[,-1]
colnames(sn2)<-c("value","from","to")
sn2<-sn2[,c(2,3,1)]
sn2$pval<- -log10(aggregate(temp2$pval,list(temp2$cp),FUN=geometric.mean)[,2])
sn2$n_spot<- round(aggregate(temp2$n_spot,list(temp2$cp),FUN=mean)[,2],2)
sn2$exp<-round(aggregate(temp2$cp_lr_mean_scaled,list(temp2$cp),FUN=mean)[,2],2)
head(sn2)

In [313]:
write.csv(sn2,paste0(sf,"3h.spmc-04.pval1e-3_spot10.igraph.df.csv"))

In [ ]:
sn3<-as.data.frame(table(temp3$cp))
sn3$source<-str_split(sn3$Var1,"_",simplify = T)[,1]
sn3$target<-str_split(sn3$Var1,"_",simplify = T)[,2]
rownames(sn3)<-sn3[,1]
sn3<-sn3[,-1]
colnames(sn3)<-c("value","from","to")
sn3<-sn3[,c(2,3,1)]
sn3$pval<- -log10(aggregate(temp3$pval,list(temp3$cp),FUN=geometric.mean)[,2])
sn3$n_spot<- round(aggregate(temp3$n_spot,list(temp3$cp),FUN=mean)[,2],2)
sn3$exp<-round(aggregate(temp3$cp_lr_mean_scaled,list(temp3$cp),FUN=mean)[,2],2)
head(sn3)

In [315]:
write.csv(sn3,paste0(sf,"24h.pval1e-4_spot10.igraph.df.csv"))

In [ ]:
sn4<-as.data.frame(table(temp4$cp))
sn4$source<-str_split(sn4$Var1,"_",simplify = T)[,1]
sn4$target<-str_split(sn4$Var1,"_",simplify = T)[,2]
rownames(sn4)<-sn4[,1]
sn4<-sn4[,-1]
colnames(sn4)<-c("value","from","to")
sn4<-sn4[,c(2,3,1)]
sn4$pval<- -log10(aggregate(temp4$pval,list(temp4$cp),FUN=geometric.mean)[,2])
sn4$n_spot<- round(aggregate(temp4$n_spot,list(temp4$cp),FUN=mean)[,2],2)
sn4$exp<-round(aggregate(temp4$cp_lr_mean_scaled,list(temp4$cp),FUN=mean)[,2],2)
head(sn4)

In [318]:
write.csv(sn4,paste0(sf,"72h.pval1e-3_spot10.igraph.df.csv"))

In [322]:
nodes<-data.frame(name=c('R.Microglia','Macrophage','Homeostatic.Microglia','Astrocyte.Gfap',
                         'Vascular',#'Astro.Svep1',
                         'Ependymal','Oligodendrocyte','Fibroblast',
                         'Neuron','Astrocyte.Slc7a10','OPC','Monocyte',
                         "Neutrophil","Myeloid"
                         ))

In [ ]:
test<-sn1
#colnames(test)<-c('Var1','n','from','to')
test$n_spot<- scale(as.numeric(test$n_spot), center = 0)
        links <- data.frame(
            source = test$from,
            target = test$to,
            importance = as.numeric(test$n_spot)
          )
        #head(links)
        #nodes <- data.frame(name=unique(c(test$from,test$to)))
        #nodes
        graph_ntw <- igraph::graph_from_data_frame(links,
                vertices = nodes,
                directed = FALSE)
        
        deg <- degree(graph_ntw, mode="all")
        #pdf(paste0(result_dir,"spatial_celltype_withinspot_interaction_probability.circle.plot.png"))
        # Get color palette for difusion
        edge_importance <- E(graph_ntw)$importance #
        edge_col<-test$exp
        # Select a continuous palette
        qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'seq',]
        
        # Create a color palette
        #getPalette <- colorRampPalette(rev(c("#E99285","#F6D0CA","#F1F9AE","#CAF0F6","#85DCE9")))
        #getPalette <- colorRampPalette(rev(c("#290ab8","#aebaff","#3552ff","#0022e4","#d7cffc")))
        getPalette <- colorRampPalette(#viridisLite::viridis(n = 10,option = "G"))
                                       rev(c("#0C3321","#155637","#1a6a44","#1e7b4e","#2A6A63","#50817A","#6AA39A","#98BFB9","#C5DBD8")))

        
        #getPalette <- colorRampPalette(rev(c("#1D487C","#2864AD","#3d80d2","#6399db","#99bde7")))
        # Get how many values we need
        #grad_edge <- seq(3, 16
        #                 , 0.1)
        grad_edge <- seq(0, 1, 0.01)
        # Generate extended gradient palette dataframe
        graph_col_df <- data.frame(value = as.character(grad_edge),
                                   color = getPalette(length(grad_edge)),
                                   stringsAsFactors = FALSE)
        # Assign color to each edge
        color_edge <- data.frame(value = as.character(round(edge_col, 2)), stringsAsFactors = FALSE) %>%
          dplyr::left_join(graph_col_df, by = "value") %>%
          dplyr::pull(color)
        pdf(file=paste0(sf,"sham.spmc-04.pval1e-3_spot10.cp.lr.nspot_width.number_color.igraph.circle.plot_220725.pdf"))
        plot(graph_ntw,
             # Size of the edge
             edge.width = edge_importance*10, #*2 enhance the visualization
             edge.color = color_edge,
             # Size of the buble
             vertex.size = deg*4,
             vertex.color = "#E4791B",
             vertex.frame.color = "grey",
             vertex.label.color = "black",
             #vertex.label.family = "Ubuntu", # Font family of the label (e.g.“Times”, “Helvetica”)
             layout = layout.circle)
       dev.off()

In [ ]:
test<-sn2
#colnames(test)<-c('Var1','n','from','to')
test$n_spot<- scale(as.numeric(test$n_spot), center = 0)
        links <- data.frame(
            source = test$from,
            target = test$to,
            importance = as.numeric(test$n_spot)
          )
        #head(links)
        #nodes <- data.frame(name=unique(c(test$from,test$to)))
        #nodes
        graph_ntw <- igraph::graph_from_data_frame(links,
                vertices = nodes,
                directed = FALSE)
        
        deg <- degree(graph_ntw, mode="all")
        #pdf(paste0(result_dir,"spatial_celltype_withinspot_interaction_probability.circle.plot.png"))
        # Get color palette for difusion
        edge_importance <- E(graph_ntw)$importance #
        edge_col<-test$exp
        # Select a continuous palette
        qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'seq',]
        
        # Create a color palette
        #getPalette <- colorRampPalette(rev(c("#E99285","#F6D0CA","#F1F9AE","#CAF0F6","#85DCE9")))
        #getPalette <- colorRampPalette(rev(c("#290ab8","#aebaff","#3552ff","#0022e4","#d7cffc")))
        getPalette <- colorRampPalette(rev(c("#0C3321","#155637","#1a6a44","#1e7b4e","#2A6A63","#50817A","#6AA39A","#98BFB9","#C5DBD8")))
        #getPalette <- colorRampPalette(rev(c("#1D487C","#2864AD","#3d80d2","#6399db","#99bde7")))
        # Get how many values we need
        #grad_edge <- seq(3, 16
        #                 , 0.1)
        grad_edge <- seq(0, 1, 0.01)
        # Generate extended gradient palette dataframe
        graph_col_df <- data.frame(value = as.character(grad_edge),
                                   color = getPalette(length(grad_edge)),
                                   stringsAsFactors = FALSE)
        # Assign color to each edge
        color_edge <- data.frame(value = as.character(round(edge_col, 2)), stringsAsFactors = FALSE) %>%
          dplyr::left_join(graph_col_df, by = "value") %>%
          dplyr::pull(color)
        pdf(file=paste0(sf,"3h.spmc-04.pval1e-3_spot10.cp.lr.nspot_width.number_color.igraph.circle.plot.pdf"))
        plot(graph_ntw,
             # Size of the edge
             edge.width = edge_importance*10, #*2 enhance the visualization
             edge.color = color_edge,
             # Size of the buble
             vertex.size = deg*4,
             vertex.color = "#E4791B",
             vertex.frame.color = "grey",
             vertex.label.color = "black",
             #vertex.label.family = "Ubuntu", # Font family of the label (e.g.“Times”, “Helvetica”)
             layout = layout.circle)
       dev.off()

In [ ]:
test<-sn3
#colnames(test)<-c('Var1','n','from','to')
test$n_spot<- scale(as.numeric(test$n_spot), center = 0)
        links <- data.frame(
            source = test$from,
            target = test$to,
            importance = as.numeric(test$n_spot)
          )
        #head(links)
        #nodes <- data.frame(name=unique(c(test$from,test$to)))
        #nodes
        graph_ntw <- igraph::graph_from_data_frame(links,
                vertices = nodes,
                directed = FALSE)
        
        deg <- degree(graph_ntw, mode="all")
        #pdf(paste0(result_dir,"spatial_celltype_withinspot_interaction_probability.circle.plot.png"))
        # Get color palette for difusion
        edge_importance <- E(graph_ntw)$importance #
        edge_col<-test$exp
        # Select a continuous palette
        qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'seq',]
        
        # Create a color palette
        #getPalette <- colorRampPalette(rev(c("#E99285","#F6D0CA","#F1F9AE","#CAF0F6","#85DCE9")))
        #getPalette <- colorRampPalette(rev(c("#290ab8","#aebaff","#3552ff","#0022e4","#d7cffc")))
        getPalette <- colorRampPalette(rev(c("#0C3321","#155637","#1a6a44","#1e7b4e","#2A6A63","#50817A","#6AA39A","#98BFB9","#C5DBD8")))
        #getPalette <- colorRampPalette(rev(c("#1D487C","#2864AD","#3d80d2","#6399db","#99bde7")))
        # Get how many values we need
       # grad_edge <- seq(3, 16
       #                  , 0.1)
        
        grad_edge <- seq(0, 1, 0.01)

        # Generate extended gradient palette dataframe
        graph_col_df <- data.frame(value = as.character(grad_edge),
                                   color = getPalette(length(grad_edge)),
                                   stringsAsFactors = FALSE)
        # Assign color to each edge
        color_edge <- data.frame(value = as.character(round(edge_col, 2)), stringsAsFactors = FALSE) %>%
          dplyr::left_join(graph_col_df, by = "value") %>%
          dplyr::pull(color)
        pdf(file=paste0(sf,"24h.pval1e-3_spot10.cp.lr.nspot_width.number_color.igraph.circle.plot.pdf"))
        plot(graph_ntw,
             # Size of the edge
             edge.width = edge_importance*10, #*2 enhance the visualization
             edge.color = color_edge,
             # Size of the buble
             vertex.size = deg*2,
             vertex.color = "#E4791B",
             vertex.frame.color = "grey",
             vertex.label.color = "black",
             #vertex.label.family = "Ubuntu", # Font family of the label (e.g.“Times”, “Helvetica”)
             layout = layout.circle)
        dev.off()

In [ ]:
#remove Astro-Svep1
sn4<-sn4[sn4$from!="Astro.Svep1"&sn4$to!="Astro.Svep1",]
test<-sn4
#colnames(test)<-c('Var1','n','from','to')
test$n_spot<- scale(as.numeric(test$n_spot), center = 0)
        links <- data.frame(
            source = test$from,
            target = test$to,
            importance = as.numeric(test$n_spot)
          )
        #head(links)
        #nodes <- data.frame(name=unique(c(test$from,test$to)))
        #nodes
        graph_ntw <- igraph::graph_from_data_frame(links,
                vertices = nodes,
                directed = FALSE)
        
        deg <- degree(graph_ntw, mode="all")
        #pdf(paste0(result_dir,"spatial_celltype_withinspot_interaction_probability.circle.plot.png"))
        # Get color palette for difusion
        edge_importance <- E(graph_ntw)$importance #
        edge_col<-test$exp
        # Select a continuous palette
        qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'seq',]
        
        # Create a color palette
        #getPalette <- colorRampPalette(rev(c("#E99285","#F6D0CA","#F1F9AE","#CAF0F6","#85DCE9")))
        #getPalette <- colorRampPalette(rev(c("#290ab8","#aebaff","#3552ff","#0022e4","#d7cffc")))
        getPalette <- colorRampPalette(rev(c("#0C3321","#155637","#1a6a44","#1e7b4e","#2A6A63","#50817A","#6AA39A","#98BFB9","#C5DBD8")))
        #getPalette <- colorRampPalette(rev(c("#1D487C","#2864AD","#3d80d2","#6399db","#99bde7")))
        # Get how many values we need
       #grad_edge <- seq(3, 16
       #                 , 0.1)
        #grad_edge <- seq(11, 155, 0.1)
        grad_edge <- seq(0, 1, 0.01)
        
        # Generate extended gradient palette dataframe
        graph_col_df <- data.frame(value = as.character(grad_edge),
                                   color = getPalette(length(grad_edge)),
                                   stringsAsFactors = FALSE)
        # Assign color to each edge
        color_edge <- data.frame(value = as.character(round(edge_col, 2)), stringsAsFactors = FALSE) %>%
          dplyr::left_join(graph_col_df, by = "value") %>%
          dplyr::pull(color)
       pdf(file=paste0(sf,"72h.pval1e-3_spot10.cp.lr.nspot_width.number_color.igraph.circle.plot.pdf"))
        plot(graph_ntw,
             # Size of the edge
             edge.width = edge_importance*10, #*2 enhance the visualization
             edge.color = color_edge,
             # Size of the buble
             vertex.size = deg*4,
             vertex.color = "#E4791B",
             vertex.frame.color = "grey",
             vertex.label.color = "black",
             #vertex.label.family = "Ubuntu", # Font family of the label (e.g.“Times”, “Helvetica”)
             layout = layout.circle)
       dev.off()